In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import os
from tqdm import tqdm


In [3]:
# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

# Configure PyTorch to use the GPU
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True


Using device: cuda


In [4]:
# Define image dimensions and other parameters
image_height = 256
image_width = 256
batch_size = 256
epochs = 50
validation_split = 0.2
learning_rate = 0.0003


In [5]:
# Define the CNN architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * (image_height // 8) * (image_width // 8), 128)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 7)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool(x)
        x = torch.relu(self.conv2(x))
        x = self.pool(x)
        x = torch.relu(self.conv3(x))
        x = self.pool(x)
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.softmax(self.fc2(x), dim=1)
        return x


In [6]:
model = CNN().to(device)

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# Data preprocessing and augmentation
data_transforms = transforms.Compose([
    transforms.Resize((image_height, image_width)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop((image_height, image_width), scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [7]:
# Load images from the 'denominations' folder with subdirectories
data_dir = '/content/drive/MyDrive/denominations'
full_dataset = datasets.ImageFolder(data_dir, transform=data_transforms)

# Create train and validation split
num_train = int((1 - validation_split) * len(full_dataset))
num_val = len(full_dataset) - num_train
train_dataset, val_dataset = random_split(full_dataset, [num_train, num_val])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)


In [8]:
# Initialize best accuracy
best_val_acc = 0.0

# Training the model
for epoch in tqdm(range(epochs), desc=f"Training"):
    model.train()
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = running_corrects.double() / len(train_loader.dataset)

    print(f'\nEpoch {epoch}/{epochs - 1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

    # Validation phase
    model.eval()
    val_running_loss = 0.0
    val_running_corrects = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)

            val_running_loss += loss.item() * inputs.size(0)
            val_running_corrects += torch.sum(preds == labels.data)

    val_epoch_loss = val_running_loss / len(val_loader.dataset)
    val_epoch_acc = val_running_corrects.double() / len(val_loader.dataset)

    print(f'\nValidation Loss: {val_epoch_loss:.4f}, Validation Accuracy: {val_epoch_acc:.4f}')

    # Check if this is the best model so far
    if val_epoch_acc > best_val_acc:
        best_val_acc = val_epoch_acc
        torch.save(model.state_dict(), '/content/drive/MyDrive/FK Models/best_model.pth')
        print(f'\nBest model saved with validation accuracy: {best_val_acc:.4f}')

print(f'\nBest validation accuracy: {best_val_acc:.4f}')

Training:   0%|          | 0/50 [00:00<?, ?it/s]


Epoch 0/49, Loss: 1.9379, Accuracy: 0.1756

Validation Loss: 1.9140, Validation Accuracy: 0.2061


Training:   2%|▏         | 1/50 [00:54<44:44, 54.79s/it]


Best model saved with validation accuracy: 0.2061

Epoch 1/49, Loss: 1.8824, Accuracy: 0.2548

Validation Loss: 1.8615, Validation Accuracy: 0.2739


Training:   4%|▍         | 2/50 [01:36<37:40, 47.09s/it]

...


Best model saved with validation accuracy: 0.8603

Epoch 48/49, Loss: 1.3046, Accuracy: 0.8760


Training:  98%|█████████▊| 49/50 [34:48<00:42, 42.74s/it]


Validation Loss: 1.3281, Validation Accuracy: 0.8520

Epoch 49/49, Loss: 1.2968, Accuracy: 0.8854


Training: 100%|██████████| 50/50 [35:31<00:00, 42.63s/it]


Validation Loss: 1.3336, Validation Accuracy: 0.8313

Best validation accuracy: 0.8603
